In [1]:
# Render our plots inline
%matplotlib inline
%pylab inline  
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

# General Plotting Parameters
mpl.rcParams['figure.figsize'] = (8,5)
mpl.rcParams['lines.linewidth'] = 2.5
mpl.rcParams['font.weight'] = 'bold'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['font.size'] = 14.
mpl.rcParams['legend.fontsize'] = 12.
mpl.rcParams['axes.labelsize'] = 12.
mpl.rcParams['xtick.labelsize'] = 10.
mpl.rcParams['ytick.labelsize'] = 10.
mpl.rcParams['xtick.minor.pad'] = 4
mpl.rcParams['xtick.direction'] = 'out'
mpl.rcParams['ytick.direction'] = 'out'

#Git says this is patched, but it doesn't work from Pip --upgrade 26-mar-2015
#mpl.rcParams['xtick.minor.visible'] = True  

# These are the "Tableau 20" colors as RGB.  
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14),
             (255, 187, 120), (44, 160, 44), (152, 223, 138),
              (148, 103, 189),
             (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127),
             (199, 199, 199), (188, 189, 34), (219, 219, 141),
             (23, 190, 207), (158, 218, 229),(214, 39, 40), (255, 152, 150)]  
    
# Scale the RGB values to the [0, 1] range,
# which is the format matplotlib accepts.  
for i in range(len(tableau20)): 
    r, g, b = tableau20[i]  
    tableau20[i] = (r / 255., g / 255., b / 255.)  

Populating the interactive namespace from numpy and matplotlib


In [50]:
cut = 0.5
cascade_feed = 5
n_enrich = 4
n_strip = 0

In [51]:
n_stages = n_strip + n_enrich

eqn_answers = np.zeros(n_stages)
print eqn_answers
for i in range(-1*n_strip, n_enrich):
    eqn = np.zeros(n_stages)
    position = n_strip + i
    eqn[position] = -1
    if (position != 0):
        eqn[position - 1] = cut
    if (position != n_stages - 1):
        eqn[position + 1] = (1-cut)
    print i , position
    print eqn
    if (position == 0):
        eqn_array = eqn
    else:
        eqn_array = np.vstack((eqn_array,eqn))
    if (i == 0):
        eqn_answers[position] = -1*cascade_feed

res = np.linalg.solve(eqn_array, eqn_answers)
      
res

[ 0.  0.  0.  0.]
0 0
[-1.   0.5  0.   0. ]
1 1
[ 0.5 -1.   0.5  0. ]
2 2
[ 0.   0.5 -1.   0.5]
3 3
[ 0.   0.   0.5 -1. ]


array([ 8.,  6.,  4.,  2.])

In [52]:

a = np.array([[-1, 1-cut, 0,0],[cut, -1, 1-cut, 0],[0,cut,-1, 1-cut],[0,0,cut,-1]])
b = np.array([-Fc,0,0,0])
x = np.linalg.solve(a, b)

x

array([ 8.,  6.,  4.,  2.])